In [1]:
import os
import tensorflow as tf

gpu = tf.config.list_physical_devices('GPU')


In [2]:
gpu[0]

if gpu:
    try:
#         tf.config.experimental.set_virtual_device_configuration(gpu[0], 
#          [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
        tf.config.experimental.set_memory_growth(gpu[0], True)
    except RuntimeError as e:
        print(e)

In [3]:
%cd C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn/Mask_RCNN

C:\Users\user\Desktop\CV\3\image-seg-mask-rcnn\Mask_RCNN


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import skimage
import os
import sys
import json
import datetime
import numpy as np
import cv2

In [6]:
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

In [7]:
from mrcnn.config import Config
from mrcnn import model as modellib, utils

In [8]:
#root director 
ROOT_DIR =  "C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn"

#TRAINED WEIGHTS FILE
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "input/mask_rcnn_coco.h5")

#logs directory
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "output/logs")

In [9]:
class CustomConfig(Config):
    "Configuring training on the dataset. Builds on the base Config class and overides some values"
    
    #configuration name
    NAME = "object"
    
    GPU_COUNT = 1
    
    #set images per gpu
    IMAGES_PER_GPU = 1
    
    #no of classes
    NUM_CLASSES = 1 +  1
    
    STEPS_PER_EPOCH = 30
    
#     BATCH_SIZE = 1
    
    #skip detections with  <90% of confidence
    DETECTION_MIN_CONFIDENCE = 0.5

In [10]:
class CustomDataset(utils.Dataset):
    
    def load_custom(self, dataset_dir, subset):
        #dataset_dir = root directory
        #subset = train or val
        
        self.add_class("object", 1, "tumor")
        
        #train or val
        assert subset in ["train_main", "val_main"]
        dataset_dir = os.path.join(dataset_dir, subset)
        print(dataset_dir)
        
        annotations1 = json.load(open(os.path.join(dataset_dir, "annotations_fill.json" )))
        annotations = list(annotations1.values())
        
        #skip images without annotations
        annotations = [a for a in annotations if a['regions']]
        
        #adding images
        for a in annotations:
#             print(a)
            #get x,y coordinates of points in the polygon thatmake up the object outline
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['tumor'] for s in a['regions']]
#   
            print("objects:", objects)
            
            name_dict = {"tumor": 1}
            print(objects)
            num_ids = [name_dict[a] for  a in objects]
            
            print("numids", num_ids)
            image_path = os.path.join(dataset_dir, a['filename'] )
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]
            
            self.add_image(
                    "object", #adding the name to each class
                    image_id = a['filename'],
                    path = image_path,
                    width = width, height=height,
                    polygons=polygons,
                    num_ids=num_ids
                )
            
    def load_mask(self, image_id):
        #Generating instance masks for images
        #returns: an array of boolean values of shape [height, width, instance count]
        #class_ids: one dim array of class ids of the instance masks
        
        #if the image is not an object type, delegate to parent class
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self._class_, self).load_mask(image_id)
        
        #convert polygons to a bitmap mask of shape
        #[height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                       dtype=np.uint8)
        
        for i, p in enumerate(info["polygons"]):
            #set indexes of pixels inside the polygon to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            
            mask[rr, cc, i] =1
            
            #return mask and array of class ids of each instance.our class is one so we return one class id
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids
        
    def image_reference(self, image_id):
        "return image path"
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        
        else:
            super(self.__class__, self).image_reference(image_id)

In [11]:
def train(model):
    
    train_dataset = CustomDataset()
    train_dataset.load_custom(ROOT_DIR+"/input/data/TRAIN", "train_main")
    train_dataset.prepare()
    
    #validation
    validation_data = CustomDataset()
    validation_data.load_custom(ROOT_DIR+"/input/data/VAL", "val_main")
    validation_data.prepare()
    
    print("Training network")
    return model.train(train_dataset, validation_data, 
                      learning_rate = config.LEARNING_RATE,
                      epochs=10,
                      layers='heads')

In [12]:
from tensorflow.python.keras.saving import hdf5_format

config = CustomConfig()

model = modellib.MaskRCNN(mode="training", config=config,
                         model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH

print(weights_path)
model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits",
                                                       "mrcnn_bbox_fc",
                                                       "mrcnn_bbox",
                                                       "mrcnn_mask"])

C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn\input/mask_rcnn_coco.h5


In [ ]:
train(model)

C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn/input/data/TRAIN\train_main
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor', 'tumor']
['tumor', 'tumor']
numids [1, 1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor', 'tumor']
['tumor', 'tumor']
numids [1, 1]
objects: ['tumor', 'tumor']
['tumor', 'tumor']
numids [1, 1]
objects: ['tumor', 'tumor']
['tumor', 'tumor']
numids [1, 1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor', 'tumor']
['tumor', 'tumor']
numids [1, 1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor', 'tumor', 'tumor']
['tumor', 'tumor', 'tumor']
numids [1, 1, 1]
objects: ['tumor']
['tumor']
numids [1]
objects: ['tumor']

In [ ]:
# import json

# pattern = r'"region_attributes":{}'
# replacement = '"region_attributes":{"tumor":"tumor"}'

# output_file = "C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn/input/data/TRAIN/train_main/annotations_fill.json"

# input_file = "C:/Users/user/Desktop/CV/3/image-seg-mask-rcnn/input/data/TRAIN/train_main/annotations.json"

# with open(input_file, 'r') as f:
#     data = f.read()
    
# new_data = re.sub(pattern, replacement, data)

# with open(output_file, 'w') as output:
#     output.write(new_data)
    

               
                
            